In [1]:
# Dependencies
import tweepy
import numpy as np
import pandas as pd

# Import and Initialize Sentiment Analyzer
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
analyzer = SentimentIntensityAnalyzer()

# Twitter API Keys
from config import (consumer_key, 
                    consumer_secret, 
                    access_token, 
                    access_token_secret)

# Setup Tweepy API Authentication
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)
api = tweepy.API(auth, parser=tweepy.parsers.JSONParser())

In [2]:
# Target Search Term
target_terms = ("@SouthwestAir", "@AmericanAir", "@SpiritAirlines",
                "@Virginatlantic", "@Delta", "@AlaskaAir", "@KLM")

# "Real Person" Filters
min_tweets = 5
max_tweets = 10000
max_followers = 2500
max_following = 2500
lang = "en"

# List to hold results
results_list = []

# Loop through all target users
for target in target_terms:

    # Variable for holding the oldest tweet
    oldest_tweet = None

    # Variables for holding sentiments
    compound_list = []
    positive_list = []
    negative_list = []
    neutral_list = []

    # Loop through 10 times
    for x in range(10):

        # Run search around each tweet
        public_tweets = api.search(
            target, count=100, result_type="recent", max_id=oldest_tweet)

        # Loop through all tweets
        for tweet in public_tweets["statuses"]:

            # Use filters to check if user meets conditions
            if (tweet["user"]["followers_count"] < max_followers
                and tweet["user"]["statuses_count"] > min_tweets
                and tweet["user"]["statuses_count"] < max_tweets
                and tweet["user"]["friends_count"] < max_following
                and tweet["user"]["lang"] == lang):

                # Run Vader Analysis on each tweet
                results = analyzer.polarity_scores(tweet["text"])
                compound = results["compound"]
                pos = results["pos"]
                neu = results["neu"]
                neg = results["neg"]

                # Add each value to the appropriate list
                compound_list.append(compound)
                positive_list.append(pos)
                negative_list.append(neg)
                neutral_list.append(neu)
                
            # Set the new oldest_tweet value
            oldest_tweet = tweet["id"] - 1

    # Store the Average Sentiments
    sentiment = {
        "User": target,
        "Compound": np.mean(compound_list),
        "Positive": np.mean(positive_list),
        "Neutral": np.mean(negative_list),
        "Negative": np.mean(neutral_list),
        "Tweet Count": len(compound_list)
    }

    # Print the Sentiments
    print(sentiment)
    print()
    
    # Append airline results to 'results_list'
    results_list.append(sentiment)

{'User': '@SouthwestAir', 'Compound': 0.14046563467492262, 'Positive': 0.12346904024767801, 'Neutral': 0.059399380804953557, 'Negative': 0.8171362229102167, 'Tweet Count': 646}

{'User': '@AmericanAir', 'Compound': 0.100359793814433, 'Positive': 0.10965773195876288, 'Neutral': 0.058777319587628871, 'Negative': 0.83154845360824736, 'Tweet Count': 485}

{'User': '@SpiritAirlines', 'Compound': 0.045246522781774587, 'Positive': 0.095115107913669075, 'Neutral': 0.071261390887290166, 'Negative': 0.83361390887290154, 'Tweet Count': 417}

{'User': '@Virginatlantic', 'Compound': 0.15288962962962963, 'Positive': 0.11706111111111113, 'Neutral': 0.046201851851851848, 'Negative': 0.83673703703703706, 'Tweet Count': 540}

{'User': '@Delta', 'Compound': -0.01190765957446809, 'Positive': 0.11145106382978723, 'Neutral': 0.090870212765957456, 'Negative': 0.79767659574468086, 'Tweet Count': 470}

{'User': '@AlaskaAir', 'Compound': 0.24751421143847485, 'Positive': 0.14214211438474872, 'Neutral': 0.0384384

In [3]:
airline_df = pd.DataFrame(results_list).set_index("User").round(3)
airline_df

,Compound,Negative,Neutral,Positive,Tweet Count
User,,,,,
@SouthwestAir,0.140,0.817,0.059,0.123,646
@AmericanAir,0.100,0.832,0.059,0.110,485
@SpiritAirlines,0.045,0.834,0.071,0.095,417
@Virginatlantic,0.153,0.837,0.046,0.117,540
@Delta,-0.012,0.798,0.091,0.111,470
@AlaskaAir,0.248,0.819,0.038,0.142,577
@KLM,0.012,0.821,0.076,0.103,419
